In [ ]:
import numpy as np
import pandas as pd
import requests
import re
import json
from os import walk
from multiprocessing.pool import ThreadPool

In [ ]:
URL = "https://s3.amazonaws.com/irs-form-990/index_2014.json"

response = requests.get(URL)
with open("index_2014.json", 'wb') as f:
    f.write(response.content)

In [ ]:
with open("index_2014.json") as f:
    data = json.load(f)
    data = data[list(data.keys())[0]]

In [ ]:
df = pd.DataFrame.from_dict(data)
df.head(5)

In [ ]:
df.shape

In [ ]:
def download_url(url):
    file_name_start = url.rfind('/') + 1
    file_name = url[file_name_start:]
    output_dir = "/home/meso/git_repo/irs990/data/data_2014/"
    file_name = output_dir + file_name
    
    r = requests.get(url, stream=True)
    if r.status_code == requests.codes.ok:
        with open(file_name, 'wb') as f:
            f.write(r.content)
    return url

urls = []
for i in range(df.shape[0]): 
    urls.append(data[i]['URL'])
    
results = ThreadPool(100).imap_unordered(download_url, urls)

In [ ]:
f = []
for (dirpath, dirnames, filenames) in walk('/home/meso/git_repo/irs990/data/data_2014/'):
    f.extend(filenames)
    break

In [ ]:
u = list(df['URL'])
len(u)

In [ ]:
file_names_list = []

for i in range(len(u)):
    file_loc = u[i].rfind("/") + 1
    file_name = u[i][file_loc:]
    file_names_list.append(file_name)

In [ ]:
print(len(file_names_list), str(":"), len(f))

In [ ]:
diff = list(set(file_names_list) - set(f))
len(diff)

In [ ]:
file_loc = u[i].rfind("/") + 1
url_pre = u[0][:file_loc]

In [ ]:
urls = []
for i in range(len(diff)):
    url = url_pre + diff[i]
    urls.append(url)

In [ ]:
for url in urls:
    download_url(url)

In [ ]:
f = []
for (dirpath, dirnames, filenames) in walk('/home/meso/git_repo/irs990/data/data_2014/'):
    f.extend(filenames)
    break

In [ ]:
len(f)

In [ ]:
diff = list(set(file_names_list) - set(f))
len(diff)

In [ ]:
def list_duplicates(seq):
    seen = set()
    seen_add = seen.add
    return [idx for idx, item in enumerate(seq) if item in seen or seen_add(item)]

In [ ]:
print(list_duplicates(file_names_list))

In [ ]:
dup_df = df.iloc[list_duplicates(file_names_list)]

In [ ]:
dup_ein = pd.DataFrame(dup_df['EIN'])

In [ ]:
dup_ein

In [ ]:
duplicates = pd.DataFrame([])

for dup in range(dup_ein.shape[0]):
    ein = dup_ein.iloc[dup]['EIN']
    duplicates = duplicates.append(df[df['EIN'] == ein])

In [ ]:
duplicates